In [1]:
!pip install pandas openpyxl openai==1.34.0

In [2]:
import pandas as pd

excel_data = pd.read_excel('data/Data for ML Test.xlsx', sheet_name=None)

examples_data_df  = excel_data["Example Sheet"]
to_do_df  = excel_data["To Do"]

In [4]:
examples_data_df.head()

,S.No,Suppler Name,Suppler Parent
0,1,IBM CORPORATION,IBM CORPORATION-
1,2,IBM CORP,IBM CORPORATION-
2,3,IBM CREDIT CORPORATION,IBM CORPORATION-
3,4,RED HAT INC,IBM CORPORATION-
4,5,RED HAT NC,IBM CORPORATION-


In [5]:
unique_parent_company = list(examples_data_df['Suppler Parent'].str.replace("-",'').unique())
unique_parent_company

['IBM CORPORATION',
 'DELL TECHNOLOGIES',
 'UNITED PARCEL SERVICE',
 'AON',
 'MICROSOFT',
 'UNITEDHEALTH GROUP',
 'WELLS FARGO FINANCIAL SERVICES']

In [7]:
to_do_df.head()

,S.No,Supplier Name,Supplier Parent
0,1,WELLCARE HEALTH PLANS,NaN
1,2,BANK OF AMERICA,NaN
2,3,MCKESSON,NaN
3,4,DIGITASLBI,NaN
4,5,OUTCOMESMTM,NaN


In [8]:
supplier_name_list:list = to_do_df["Supplier Name"].tolist()

In [ ]:
def update_unique_parent(parent_name):
    if not parent_name in unique_parent_company:
        unique_parent_company.append(parent_name)
    return 

In [9]:
def get_prompt(supplier_name: str):
    # parent_list =','.join(unique_parent_company)
    return f"""Find out the parent company of the specified subsidiary company. 
The response should be a JSON object with the key name 'parent'.
---
examples:
user: RED HAT INC    
assistant: {{"parent":"IBM CORPORATION"}}

user: BOOMI INC        
assistant: {{"parent":"DELL TECHNOLOGIES"}}

user: {supplier_name}
assistant: """


In [10]:
import os
from getpass import getpass

azure_openai_endpoint = getpass("Enter Azure OpenAI Endpoint: ")
azure_openai_deployment = getpass("Enter Azure OpenAI Deployment Name: ")
azure_openai_key = getpass("Enter Azure OpenAI API Key: ")

os.environ["AZURE_OPENAI_ENDPOINT"] = azure_openai_endpoint
os.environ["DEPLOYMENT_NAME"] = azure_openai_deployment
os.environ["AZURE_OPENAI_KEY"] = azure_openai_key

In [11]:
import os
from openai import AzureOpenAI

azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
deployment_name = os.environ["DEPLOYMENT_NAME"]
api_key = os.environ["AZURE_OPENAI_KEY"]
api_version = "2024-02-01"

def llm(prompt:str):

    client = AzureOpenAI(
            azure_endpoint=azure_endpoint,
            api_key=api_key,
            api_version=api_version
        )
    completion = client.chat.completions.create(
        model=deployment_name,  # model = "deployment_name"
        messages= [
        {"role": "user", "content": prompt}],
        temperature=0.5,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return completion.choices[0].message.content

In [12]:
import json

def get_parent_company(*supplier_names):
    parent_company:list = []
    for each_supplier in supplier_names:
        out = llm(get_prompt(each_supplier))
        try:
            json_data = json.loads(out)
            json_data["raw"] = out
            update_unique_parent(json_data['parent'])
        except Exception as e:
            json_data =  {"raw": out,"parent": " "}
        parent_company.append(json_data)
        # print(f"S: {each_supplier} :: P: {json_data}")
    return parent_company

In [13]:
out_parent_company_list = get_parent_company(*supplier_name_list)

In [28]:
to_do_df['raw'] = None
for i in range(len(out_parent_company_list)):
    parent = out_parent_company_list[i]['parent']
    if parent:
        to_do_df.at[i,'Supplier Parent'] = parent
    else:
        to_do_df.at[i,'raw'] = out_parent_company_list[i]['raw']

In [29]:
to_do_df.to_excel('data/output_after_llm_to_do.xlsx', index=False)  

In [41]:
import pandas as pd

df = pd.read_excel('data/output_after_llm_to_do.xlsx')


In [43]:
df.head()

,S.No,Supplier Name,Supplier Parent,raw
0,1,WELLCARE HEALTH PLANS,CENTENE CORPORATION,NaN
1,2,BANK OF AMERICA,BANK OF AMERICA CORPORATION,NaN
2,3,MCKESSON,MCKESSON CORPORATION,NaN
3,4,DIGITASLBI,PUBLICIS GROUPE,NaN
4,5,OUTCOMESMTM,COTIVITI HOLDINGS INC,NaN


In [47]:
df

,S.No,Supplier Name,Supplier Parent,raw
0,1,WELLCARE HEALTH PLANS,CENTENE CORPORATION,NaN
1,2,BANK OF AMERICA,BANK OF AMERICA CORPORATION,NaN
2,3,MCKESSON,MCKESSON CORPORATION,NaN
3,4,DIGITASLBI,PUBLICIS GROUPE,NaN
4,5,OUTCOMESMTM,COTIVITI HOLDINGS INC,NaN
...,...,...,...,...
1529,1530,KIMCO HORSHAM LP SPAH1337A-LCVS///0,KIMCO REALTY CORPORATION,NaN
1530,1531,PNC BANK 1008175985 HERMAN LIPSITZ,PNC FINANCIAL SERVICES GROUP,NaN
1531,1532,TAYLOR CORPORATION,TAYLOR CORPORATION,NaN
1532,1533,LEXISNEXIS RISK SOLUTIONS INC,RELX GROUP,NaN
